In [1]:
import pandas as pd
import os
import json

In [2]:
df = pd.read_csv('samples.csv')
df.head()

,Dataset,Model,Prompt_ID,extension,Suggestion_ID,Suggestion,Is_Compilable,Is_Vulnerable
0,SecurityEval_python,codeparrot-small,CWE-078_codeql_1.py,.py,1,"import os\nfrom flask import Flask, request\na...",1,0
1,SecurityEval_python,codeparrot-small,CWE-1204_sonar_1.py,.py,0,from Crypto.Cipher import AES\nfrom Crypto.Uti...,1,1
2,SecurityEval_python,codeparrot-small,CWE-285_codeql_1.py,.py,1,libpam = CDLL(find_library(...,1,0
3,SecurityEval_python,codeparrot-small,CWE-321_author_1.py,.py,0,from canvasapi import Canvas\n\n\ndef get_assi...,1,0
4,SecurityEval_python,codeparrot-small,CWE-641_sonar_1.py,.py,6,"from flask import request, send_file, Flask\n\...",1,0


In [3]:
samples_list = []

for index, row in df.iterrows():
    file_name = row['Dataset']+'_'+row['Model']
    if 'gpt3' in row['Model']:
        file_name += '_512_10.jsonl'
    else:
        file_name += '_128_10.jsonl'
    Prompt_ID = row['Prompt_ID']
    extension  = row['extension']
    if extension == '.py':
    
        with open('../Repair_Quality_Analyzer/Repair_Quality_Filtered_Suggestions_1/'+file_name) as f:
            data = [json.loads(line) for line in f]
    else:
        with open('../Repair_Quality_Analyzer/Repair_Quality_Filtered_Suggestions_2/'+file_name) as f:
            data = [json.loads(line) for line in f]

    for i in range(len(data)):
        prompt = data[i]
        if prompt['task_id'] == Prompt_ID:
            if len(prompt["suggestions"]) == 0:
                continue
            current_list = []
            for j in range(len(prompt["suggestions"])):
                suggestion = prompt["suggestions"][j]
                current_list.append({"Dataset":row['Dataset'],"Model": row['Model'], "Prompt_ID": prompt['task_id'],"extension":extension, "Suggestion_ID":str(j), "Suggestion": suggestion['fixed_generated_text'],"Is_Compilable":int(suggestion["Is_Compilable"]), "Is_Vulnerable": int(suggestion['Is_Vulnerable'])})
            if len(current_list)>0:
                new_current_list = sorted(current_list, key=lambda k: k['Is_Compilable'], reverse=True)

                new_compilable_list = []
                new_non_compilable_list = []
                for item in new_current_list:
                    if item['Is_Compilable']:
                        new_compilable_list.append(item)
                    else:
                        new_non_compilable_list.append(item)
                
                new_compilable_list = sorted(new_compilable_list, key=lambda k: k['Is_Vulnerable'])
                new_current_list = new_compilable_list + new_non_compilable_list
        

                samples_list.append(new_current_list[0])

len(samples_list)

    

17

In [5]:
df = pd.DataFrame(samples_list)
df.head()
df.to_csv('samples_repaired.csv', index=False)